<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/02_rgb_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Prepare utils file

!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-27 01:55:42--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12110 (12K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.83K  --.-KB/s    in 0s      

2022-06-27 01:55:42 (105 MB/s) - ‘utils.py’ saved [12110/12110]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [5]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

from utils import ModelCreator, get_train_val_ds, ErrorAnalyzer, get_class_weight

In [4]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/feature_vector/2'

In [6]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [7]:
# Get shuffled train and validation dataset with one-hot labels

train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 16580 files belonging to 16 classes.
Found 3508 files belonging to 16 classes.


In [8]:
# Get class weights which have calculated from the previous notebook and stored in github

classes, class_weight = get_class_weight()

In [ ]:
for color, weight in zip(classes, class_weight.values()):
  print(color, "    \t=>", weight)

beige     	=> 17.94
black     	=> 13.8
blue     	=> 15.47
brown     	=> 17.92
cream     	=> 56.81
crimson     	=> 76.43
gold     	=> 56.81
green     	=> 21.2
grey     	=> 12.41
navy-blue     	=> 58.77
orange     	=> 22.37
red     	=> 16.39
silver     	=> 20.17
titanium     	=> 56.81
white     	=> 2.97
yellow     	=> 20.68


In [9]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [10]:
rgb_creator = ModelCreator(HUB_URL, 'RGB-model')
rgb_model = rgb_creator.make_model(img_size=IMG_SIZE)

Model: "RGB-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1536)              12930622  
                                                                 
 dense (Dense)               (None, 16)                24592     
                                                                 
Total params: 12,955,214
Trainable params: 12,845,998
Non-trainable params: 109,216
_________________________________________________________________


In [11]:
# Callbacks are EarlyStopping and ModelCheckpoint

callbacks = rgb_creator.get_callbacks()

In [12]:
history = rgb_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=15,
                        validation_data=val_ds,
                        class_weight=class_weight)

Epoch 1/15
519/519 [==============================] - 2689s 5s/step - loss: 15.0181 - auc: 0.7889 - accuracy: 0.7040 - val_loss: 1.0045 - val_auc: 0.7907 - val_accuracy: 0.7178
Epoch 2/15
519/519 [==============================] - 197s 376ms/step - loss: 9.1691 - auc: 0.8910 - accuracy: 0.8104 - val_loss: 0.6810 - val_auc: 0.8701 - val_accuracy: 0.7834
Epoch 3/15
519/519 [==============================] - 189s 363ms/step - loss: 6.8658 - auc: 0.9288 - accuracy: 0.8548 - val_loss: 0.8694 - val_auc: 0.8353 - val_accuracy: 0.7603
Epoch 4/15
519/519 [==============================] - 190s 364ms/step - loss: 5.4483 - auc: 0.9433 - accuracy: 0.8747 - val_loss: 0.7391 - val_auc: 0.8682 - val_accuracy: 0.8013
Epoch 5/15
519/519 [==============================] - 191s 365ms/step - loss: 3.8624 - auc: 0.9683 - accuracy: 0.9125 - val_loss: 0.7407 - val_auc: 0.8815 - val_accuracy: 0.8176
Epoch 6/15
519/519 [==============================] - 191s 366ms/step - loss: 2.9939 - auc: 0.9775 - accuracy: 

In [13]:
history = rgb_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=20,
                        initial_epoch=15,
                        validation_data=val_ds,
                        class_weight=class_weight)

Epoch 16/20
519/519 [==============================] - 195s 374ms/step - loss: 0.3051 - auc: 0.9989 - accuracy: 0.9923 - val_loss: 1.3207 - val_auc: 0.8238 - val_accuracy: 0.8224
Epoch 17/20
519/519 [==============================] - 188s 361ms/step - loss: 0.4224 - auc: 0.9977 - accuracy: 0.9900 - val_loss: 1.2046 - val_auc: 0.8337 - val_accuracy: 0.8312
Epoch 18/20
519/519 [==============================] - 194s 372ms/step - loss: 0.1779 - auc: 0.9995 - accuracy: 0.9959 - val_loss: 1.2166 - val_auc: 0.8304 - val_accuracy: 0.8261
Epoch 19/20
519/519 [==============================] - 188s 361ms/step - loss: 0.1403 - auc: 0.9993 - accuracy: 0.9970 - val_loss: 1.4170 - val_auc: 0.8114 - val_accuracy: 0.8147
Epoch 20/20
519/519 [==============================] - 188s 361ms/step - loss: 0.2190 - auc: 0.9991 - accuracy: 0.9941 - val_loss: 1.4189 - val_auc: 0.8231 - val_accuracy: 0.8284


In [14]:
!cp ./logs/checkpoints/RGB-model.h5 ./drive/MyDrive/checkpoints/rgb-model.h5